In [89]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


/Users/pedroperrusi/Desktop/github/simpleITK_Sandbox


 # Lamieux Method For Brain Image Segmentation
 
Replication of the results reported by Lamieux et al. : Fast, accurate, and reproducible automatic segmentation of the brain in T1-weighted volume MRI data.

 ### Initial Setup

In [91]:
from __future__ import print_function

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from ipywidgets import interact, FloatSlider

import SimpleITK as sitk
import numpy as np

from myshow import myshow, myshow3d


 Loading data and initial visualization...

In [92]:
img_T1 = sitk.ReadImage("../data/IXI002-Guys-0828-MPRAGESEN_-s256_-0301-00003-000001-01.nii")

# To visualize the labels image in RGB with needs a image with 0-255 range
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)

myshow3d(img_T1)


interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

 ## Function definitions....

In [93]:
def OstsuThresh(img):
    otsuFilter = sitk.OtsuThresholdImageFilter()
    otsuFilter.SetOutsideValue(255)
    otsuFilter.SetInsideValue(0)
    mask = otsuFilter.Execute(img)
    return mask, otsuFilter.GetThreshold()

def Threshold(img, threshValue):
    thresh = sitk.BinaryThresholdImageFilter()
    # defines threshold window
    thresh.SetUpperThreshold(threshValue)
    thresh.SetLowerThreshold(0)
    # defines inner and outer values
    thresh.SetOutsideValue(255)
    thresh.SetInsideValue(0)
    # apply threshold
    return thresh.Execute(img)

''' Performs step 2 : Initial Brain Disconnection'''
def InitialBrainDisconnection(img, threshOtsuValue, threshMult):
    # Compute a mask based on ostu value
    threshValue = threshMult * threshOtsuValue / 3
    mask = Threshold(img, threshValue)
    # Apply Conditionnal Morphological Opening Operation
    mask = ConditionalOpening(img, mask, threshOtsuValue, kernelSize=3, kernelType=1)
    # Connectivity Components Analysis
    filter = sitk.ConnectedComponentImageFilter()
    components = filter.Execute(img, mask, True)
    # reorder componends
    filter = sitk.RelabelComponentImageFilter()
    reorderedComponents = filter.Execute(components)
    # get largest component size
    component_size = filter.GetSizeOfObjectsInPixels()[0]
    # get image largest component label (pixel value = 1)
    dst = sitk.BinaryThreshold(reorderedComponents, 1, 1)
    
    # debug...
    myshow(sitk.LabelOverlay(img_T1_255, dst), 'ThreshMult: ' + str(threshMult))
    
    return dst, component_size

''' Apply a morphological open operation to seg when img pixels are over threshValue '''
def ConditionalOpening(img, seg, threshValue, kernelSize, kernelType):
    mask = Threshold(img, threshValue)
    maskedSeg = sitk.Mask(seg, mask)
    filter = sitk.BinaryMorphologicalOpeningImageFilter()
    filter.SetKernelType(kernelType)
    filter.SetKernelRadius(kernelSize)
    dst = filter.Execute(maskedSeg)
    return dst

''' Returns true for a vector composed only by zeros and false otherwise. '''
def isAllZero(vec):
    return not np.any(vec)

''' Returns true if vector contains any zero value, false otherwise. '''
def hasAnyZero(vec):
    if np.size(np.nonzero(vec)) == np.size(vec):
        return False
    else:
        return True


 ### Step 1: Calculating the Foreground Threshold Level

In [94]:
mask, threshOtsuValue = OstsuThresh(img_T1)
I_scf = threshOtsuValue / 3.5 + 6
# To visualize the labels image in RGB with needs a image with 0-255 range
myshow(sitk.LabelOverlay(img_T1_255, mask), "Otsu Thresholding")
print('Threshold Otshu value: ', threshOtsuValue)
print('Estimated cerebrospinal fluid (CSF) intensity : ', I_scf)


interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

Threshold Otshu value:  179.0
Estimated cerebrospinal fluid (CSF) intensity :  57.142857142857146


 ## Step 2 : Initial Brain Disconnection
 Performs a loop where each connected component volume found on the image is calculated. The loop stops when the change observed in the volume is greater then 3 times the moving average of the last interactions.

In [98]:
# initialize variables
threshMult = 1
largestComponentSize = 0
# first call of the interactive function
largestComponentImg, initialComponentSize = InitialBrainDisconnection(img_T1, threshOtsuValue, threshMult)
# initialize loop variables
largestComponentSize = initialComponentSize
AverageWindowSize = 3 # equals to moving average window size
MAX_INTERACTIONS = 10 # max interactions of the loop
relativeDecreace = 0 # actual relative decreace
relativeDecreacesVec = np.zeros(AverageWindowSize) # vector of past relative decreases

''' Loops InitialBrainDisconnection ultil the reduction of brain volume is greater than 3 times the moving average of the last 3 interactions. 
If relativeDecreacesVec contains only zeroes, continue. 
'''
while(relativeDecreace <= (3 * np.mean(relativeDecreacesVec)) 
      or isAllZero(relativeDecreacesVec)):
    # updates loop break condition variables ------------------------------------------------------------
    movingAvgIdx = threshMult % AverageWindowSize  # modulo is the index of the vector
    relativeDecreacesVec[movingAvgIdx] = relativeDecreace # add last decrease to past decreases vector
    lastComponetSize = largestComponentSize # updates last component size
    threshMult = threshMult + 1 # updates thresh multiplier
    ''' If maximum interactions its overcome, exit loop '''
    if threshMult >= MAX_INTERACTIONS:
        break
    # calls initial brain disconnection
    largestComponentImg, largestComponentSize = InitialBrainDisconnection(img_T1, threshOtsuValue, threshMult)
    # compute relative decrease
    relativeDecreace = np.abs(largestComponentSize - lastComponetSize)/ lastComponetSize

print('Number of interactions: ', threshMult)
myshow(sitk.LabelOverlay(img_T1_255, largestComponentImg), "Largest Component Label")

interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

2 :  0.0 >  0.0


interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

3 :  0.0 >  0.0


interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

4 :  0.11772076301136584 >  0.0


interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…

5 :  0.23353208146377555 >  0.11772076301136586
Number of interactions:  5


interactive(children=(IntSlider(value=74, description='z', max=149), Output()), _dom_classes=('widget-interact…